In [1]:
import pandas as pd
import os
import geopy.distance
import numpy as np
from pathlib import Path

In [2]:
ubike_rt = pd.read_csv('ubike_0501.csv')
ubike_rt.drop_duplicates(inplace=True)
ubike_rt['sarea'] = ubike_rt['sarea'].replace('臺大公館校區','大安區')
ubike_rt['sna'] = ubike_rt['sna'].apply(lambda x: x.split('_')[1])


In [3]:
ubike_rt = ubike_rt[ubike_rt['act']==1]

In [4]:
ubike_rt = ubike_rt[['sbi','bemp','tot','updateTime']]

In [5]:
ubike_rt['thres'] = np.min(np.column_stack(((ubike_rt['tot']*0.1).to_numpy(),np.full(len(ubike_rt),5))),axis=1)
def almost_empty(row):
    bike = row['sbi']
    thres = row['thres']
    if bike <thres:
    # if bike <2:
        return True
    else:
        return False
    
def almost_full(row):
    empty = row['bemp']
    thres = row['thres']
    if empty <thres:
    # if bike <2:
        return True
    else:
        return False

In [6]:
ubike_rt['almost_empty'] = ubike_rt.apply(almost_empty,axis=1)
ubike_rt['almost_full'] = ubike_rt.apply(almost_full,axis=1)


In [7]:
ubike_rt['date'] = pd.to_datetime(ubike_rt['updateTime']).dt.date
ubike_rt['hour'] = pd.to_datetime(ubike_rt['updateTime']).dt.hour

In [8]:
result = ubike_rt.groupby(['date','hour'],as_index=False).agg({'almost_full':'mean','almost_empty':'mean'})

In [9]:
result['dow'] = pd.to_datetime(result['date']).dt.dayofweek+1
result['weekend'] = pd.to_datetime(result['date']).dt.dayofweek.isin([5,6])
result['weekend'] = np.where(result['weekend'],'假日','平日')


In [10]:
result.to_csv('pipeline_F1_result.csv',index=False)